# 0. Install and Import Dependencies

In [1]:
%pip install mediapipe opencv-python
%pip install ultralytics


  Using cached mediapipe-0.10.7-cp310-cp310-win_amd64.whl.metadata (9.8 kB)
  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached matplotlib-3.8.1-cp310-cp310-win_amd64.whl.metadata (5.9 kB)
  Using cached numpy-1.26.1-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached opencv_contrib_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-3.20.3-cp310-cp310-win_amd64.whl (904 kB)
  Using cached sounddevice-0.4.6-py3-none-win_amd64.whl (199 kB)
  Using cached cffi-1.16.0-cp310-cp310-win_amd64.whl.metadata (1.5 kB)
  Using cached contourpy-1.2.0-cp310-cp310-win_amd64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.44.0-cp310-cp310-win_amd64.whl.metadata (156 kB)

  Using cached ultralytics-8.0.206-py3-none-any.whl.metadata (31 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached scipy-1.11.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached torch-2.1.0-cp310-cp310-win_amd64.whl.metadata (24 kB)
  Using cached torchvision-0.16.0-cp310-cp310-win_amd64.whl.metadata (6.6 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached pandas-2.1.2-cp310-cp310-win_amd64.whl.metadata (18 kB)
  Using cached seaborn-0.13.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.4-py3-none-any.whl 

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import sys
from ultralytics import YOLO
import math as m
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
#path to the yolo models
face_model = YOLO(r"face_detector.pt")
person_model = YOLO(r"yolov8n.pt")

In [5]:
# normal camera feed
cap = cv2.VideoCapture(1)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# 1. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

# 2. Calculate Angles

In [8]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

def calculate_shoulder_angle(a,b):
    a = np.array(a)
    b = np.array(b)
    
    radians = np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [9]:
def findAngle(x1, y1, x2, y2):
    theta = m.acos((y2 - y1) * (-y1) / (m.sqrt(
        (x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / m.pi) * theta
    return degree

# 3. custom pose plotting


In [1]:
 plot_pose(image,landmarks):
    fordef landmark in landmarks:
        cv2.circle(image,(int(landmark[0]),int(landmark[1])),3,(0, 0, 255),-1)
    

    cv2.line(image, (int(landmarks[8][0]),int(landmarks[8][1])), (int(landmarks[6][0]),int(landmarks[6][1])), (0, 255, 0), 2)   
    cv2.line(image, (int(landmarks[6][0]),int(landmarks[6][1])), (int(landmarks[5][0]),int(landmarks[5][1])), (0, 255, 0), 2)    
    cv2.line(image, (int(landmarks[5][0]),int(landmarks[5][1])), (int(landmarks[4][0]),int(landmarks[4][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[4][0]),int(landmarks[4][1])), (int(landmarks[0][0]),int(landmarks[0][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[0][0]),int(landmarks[0][1])), (int(landmarks[1][0]),int(landmarks[1][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[1][0]),int(landmarks[1][1])), (int(landmarks[2][0]),int(landmarks[2][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[2][0]),int(landmarks[2][1])), (int(landmarks[3][0]),int(landmarks[3][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[3][0]),int(landmarks[3][1])), (int(landmarks[7][0]),int(landmarks[7][1])), (0, 255, 0), 2)  


    #mouth line
    cv2.line(image, (int(landmarks[10][0]),int(landmarks[10][1])), (int(landmarks[9][0]),int(landmarks[9][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[9][0]),int(landmarks[9][1])), (int(landmarks[10][0]),int(landmarks[10][1])), (0, 255, 0), 2)  
  
    #main torse
    cv2.line(image, (int(landmarks[12][0]),int(landmarks[12][1])), (int(landmarks[11][0]),int(landmarks[11][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[12][0]),int(landmarks[12][1])), (int(landmarks[24][0]),int(landmarks[24][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[24][0]),int(landmarks[24][1])), (int(landmarks[23][0]),int(landmarks[23][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[11][0]),int(landmarks[11][1])), (int(landmarks[23][0]),int(landmarks[23][1])), (0, 255, 0), 2)  

    #legs
    cv2.line(image, (int(landmarks[24][0]),int(landmarks[24][1])), (int(landmarks[26][0]),int(landmarks[26][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[26][0]),int(landmarks[26][1])), (int(landmarks[28][0]),int(landmarks[28][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[23][0]),int(landmarks[23][1])), (int(landmarks[25][0]),int(landmarks[25][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[25][0]),int(landmarks[25][1])), (int(landmarks[27][0]),int(landmarks[27][1])), (0, 255, 0), 2)  
    
    #foot1
    cv2.line(image, (int(landmarks[28][0]),int(landmarks[28][1])), (int(landmarks[32][0]),int(landmarks[32][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[32][0]),int(landmarks[32][1])), (int(landmarks[30][0]),int(landmarks[30][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[28][0]),int(landmarks[28][1])), (int(landmarks[30][0]),int(landmarks[30][1])), (0, 255, 0), 2)  

    #foot2
    cv2.line(image, (int(landmarks[27][0]),int(landmarks[27][1])), (int(landmarks[29][0]),int(landmarks[29][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[29][0]),int(landmarks[29][1])), (int(landmarks[31][0]),int(landmarks[31][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[31][0]),int(landmarks[31][1])), (int(landmarks[27][0]),int(landmarks[27][1])), (0, 255, 0), 2)  

    #hand1
    cv2.line(image, (int(landmarks[12][0]),int(landmarks[12][1])), (int(landmarks[14][0]),int(landmarks[14][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[14][0]),int(landmarks[14][1])), (int(landmarks[16][0]),int(landmarks[16][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[16][0]),int(landmarks[16][1])), (int(landmarks[18][0]),int(landmarks[18][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[20][0]),int(landmarks[20][1])), (int(landmarks[18][0]),int(landmarks[18][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[20][0]),int(landmarks[20][1])), (int(landmarks[16][0]),int(landmarks[16][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[22][0]),int(landmarks[22][1])), (int(landmarks[16][0]),int(landmarks[16][1])), (0, 255, 0), 2)  

    #hand1
    cv2.line(image, (int(landmarks[11][0]),int(landmarks[11][1])), (int(landmarks[13][0]),int(landmarks[13][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[13][0]),int(landmarks[13][1])), (int(landmarks[15][0]),int(landmarks[15][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[15][0]),int(landmarks[15][1])), (int(landmarks[21][0]),int(landmarks[21][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[15][0]),int(landmarks[15][1])), (int(landmarks[19][0]),int(landmarks[19][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[19][0]),int(landmarks[19][1])), (int(landmarks[17][0]),int(landmarks[17][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[15][0]),int(landmarks[15][1])), (int(landmarks[17][0]),int(landmarks[17][1])), (0, 255, 0), 2)  

# 4. Custom Pose Estimation 

In [ ]:
def pose_estimate(landmarks):
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

    angle = calculate_shoulder_angle(left_shoulder,right_shoulder)
    angle1 = calculate_angle(left_shoulder,right_shoulder,right_elbow)
    angle2 = calculate_angle(right_shoulder,left_shoulder,left_elbow)
            
    
            
    shoulder_posture = angle >= 0 and angle <= 10
    left_arm_posture = angle1 >= 90 and angle1 <= 124
    right_arm_posture = angle2 >= 90 and angle2 <= 124
    if shoulder_posture and left_arm_posture and right_arm_posture:
        stage = "Good"
        suggestion = None
    if shoulder_posture and not(left_arm_posture and right_arm_posture):
        stage="Bad"
        suggestion = "adjust your arm posture"
    if not(shoulder_posture) and (left_arm_posture and right_arm_posture):
        stage = "Bad"
        suggestion = "adjust your shoulder posture"
    if not(shoulder_posture) and not(left_arm_posture and right_arm_posture):
        stage = "Bad"
        suggestion = "adjust your posture"
        
    return [stage,angle,angle1,angle2]


# 4. Multi-person pose detection

### 4.1 Pose detection

In [ ]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read() 
        image = frame   
        predicted = person_model(image,stream=True)
        for r in predicted:
            boxes=r.boxes
            for box in boxes:
                x1,y1,x2,y2=box.xyxwh[0]
                x1,y1,x2,y2=int(x1), int(y1), int(x2), int(y2)
                cropped_image = image[y1:y2,x1:x2]
                cropped_image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                cropped_image.flags.writeable = False
                cv2.rectangle(frame, (x1,y1), (x2,y2), (255,0,255),3)
                cropped_image = np.ascontiguousarray(cropped_image)
                results = pose.process(cropped_image)
                try:
                    landmarks = results.pose_landmarks.landmark
                    adjusted_landmarks = [(lm.x1 * x2 + x1, lm.y1 *y2  + y1) for lm in results.pose_landmarks.landmark]
                    plot_pose(image,adjusted_landmarks)
                except Exception as e:
                    print(e)
                    pass
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

<img src="results\mutiple person pose.png" style="width:300px;height:440px;margin:auto">
<img src="results\mutiple pose estimation.jpg" style="width:650px;height:440px;margin:auto">

### 5. Front and side combined version

In [15]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
stage = None
suggestion = None
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        h, w = frame.shape[:2]

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            # Calculate angle
            angle = calculate_shoulder_angle(left_shoulder,right_shoulder)
            angle1 = calculate_angle(left_shoulder,right_shoulder,right_elbow)
            angle2 = calculate_angle(right_shoulder,left_shoulder,left_elbow)
            
            
            # Visualize angle  
            cv2.putText(image, str(round(angle,3)), 
                           tuple(np.multiply(left_shoulder, [450, 450]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(round(angle1,3)), 
                           tuple(np.multiply(right_shoulder, [600, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(round(angle2,3)), 
                           tuple(np.multiply(left_shoulder, [600, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            #drawing joints
            cv2.line(image, (int(left_shoulder[0]*w), int(left_shoulder[1]*h)), (int(right_shoulder[0]*w),int(right_shoulder[1]*h)), (127, 255, 0), 2)
            cv2.line(image, (int(left_shoulder[0]*w), int(left_shoulder[1]*h)), (int(left_elbow[0]*w),int(left_elbow[1]*h)), (127, 255, 0), 2)
            cv2.line(image, (int(right_shoulder[0]*w),int(right_shoulder[1]*h)),(int(right_elbow[0]*w), int(right_elbow[1]*h)), (127, 255, 0), 2)
            
            shoulder_posture = angle >= 0 and angle <= 10
            left_arm_posture = angle1 >= 90 and angle1 <= 124
            right_arm_posture = angle2 >= 90 and angle2 <= 124
            if shoulder_posture and left_arm_posture and right_arm_posture:
                stage = "Good"
                suggestion = None
            if shoulder_posture and not(left_arm_posture and right_arm_posture):
                if(left_arm_posture):
                    cv2.line(image, (int(left_elbow[0]*w), int(left_elbow[1]*h)), (int(left_shoulder[0]*w),int(left_shoulder[1]*h)), (50, 50, 255), 2)
                if(right_arm_posture):
                    cv2.line(image, (int(right_elbow[0]*w), int(right_elbow[1]*h)), (int(right_shoulder[0]*w),int(right_shoulder[1]*h)), (50, 50, 255), 2)
                if(not(left_arm_posture or right_arm_posture)):
                    cv2.line(image, (int(left_elbow[0]*w), int(left_elbow[1]*h)), (int(left_shoulder[0]*w),int(left_shoulder[1]*h)), (50, 50, 255), 2)
                    cv2.line(image, (int(right_elbow[0]*w), int(right_elbow[1]*h)), (int(right_shoulder[0]*w),int(right_shoulder[1]*h)), (50, 50, 255), 2)
                stage="Bad"
                suggestion = "adjust your arm posture"
            if not(shoulder_posture) and (left_arm_posture and right_arm_posture):
                cv2.line(image, (int(left_shoulder[0]*w), int(left_shoulder[1]*h)), (int(right_shoulder[0]*w),int(right_shoulder[1]*h)), (50, 50, 255), 2)
                stage = "Bad"
                suggestion = "adjust your shoulder posture"
            if not(shoulder_posture) and not(left_arm_posture and right_arm_posture):
                stage = "Bad"
                suggestion = "adjust your posture"
 

            
        except:
            pass
        
         # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        cv2.putText(image, stage, 
                    (20,35), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image, suggestion, 
                    (20,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1, cv2.LINE_AA)
        
        
            
        # Render detections
#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
#                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
#                                  )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

<img src="results\IMG-20231106-WA0011.jpg">

# 6. Stress detection

### 6.1 utility function


In [ ]:
def preprocess_image(image):
    image = cv2.resize(image, (224, 224))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    return image

### 6.2 Stress detection

In [ ]:
import cv2
import tensorflow as tf
import numpy as np
from ultralytics import YOLO
import tensorflow_hub as hub

if __name__ == "__main__":
    model_path = "stress_detector.h5"
    model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

    cap = cv2.VideoCapture(1)


    while True:
        ret, frame = cap.read()
        results = face_model(frame, stream = True)
        for r in results:
            boxes=r.boxes
            for box in boxes:
                x1,y1,x2,y2=box.xyxy[0]
                x1,y1,x2,y2=int(x1), int(y1), int(x2), int(y2)
                cv2.rectangle(frame,[x1,y1],[x2,y2],(255,0,0),2)
                cropped_frame = frame[y1:y2,x1:x2]
                processed_frame = preprocess_image(cropped_frame)
                prediction = model.predict(processed_frame)
                label = "not stressed" if prediction[0][0] > 0.5 else "stressed"
                cv2.putText(frame, label, (x1,y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<img src="results\IMG-20231106-WA0010.jpg" style="width:500px;height:500px">

# 7. Stress and posture combined

In [1]:
import cv2
import tensorflow as tf
import numpy as np
import math as m
import mediapipe as mp
from ultralytics import YOLO
import tensorflow_hub as hub

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

def calculate_shoulder_angle(a,b):
    a = np.array(a)
    b = np.array(b)
    
    radians = np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

def preprocess_image(image):
    # Resize the image to match the input size of the model
    image = cv2.resize(image, (224, 224))
    # Convert the image to a format that can be input to the model
    image = np.expand_dims(image, axis=0)
    # Normalize the pixel values
    image = image / 255.0
    return image

def findAngle(x1, y1, x2, y2):
    theta = m.acos((y2 - y1) * (-y1) / (m.sqrt(
        (x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / m.pi) * theta
    return degree

def plot_pose(image,landmarks):
    for landmark in landmarks:
        cv2.circle(image,(int(landmark[0]),int(landmark[1])),3,(0, 0, 255),-1)
    cv2.line(image, (int(landmarks[8][0]),int(landmarks[8][1])), (int(landmarks[6][0]),int(landmarks[6][1])), (0, 255, 0), 2)   
    cv2.line(image, (int(landmarks[6][0]),int(landmarks[6][1])), (int(landmarks[5][0]),int(landmarks[5][1])), (0, 255, 0), 2)    
    cv2.line(image, (int(landmarks[5][0]),int(landmarks[5][1])), (int(landmarks[4][0]),int(landmarks[4][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[4][0]),int(landmarks[4][1])), (int(landmarks[0][0]),int(landmarks[0][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[0][0]),int(landmarks[0][1])), (int(landmarks[1][0]),int(landmarks[1][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[1][0]),int(landmarks[1][1])), (int(landmarks[2][0]),int(landmarks[2][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[2][0]),int(landmarks[2][1])), (int(landmarks[3][0]),int(landmarks[3][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[3][0]),int(landmarks[3][1])), (int(landmarks[7][0]),int(landmarks[7][1])), (0, 255, 0), 2)  


    #mouth line
    cv2.line(image, (int(landmarks[10][0]),int(landmarks[10][1])), (int(landmarks[9][0]),int(landmarks[9][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[9][0]),int(landmarks[9][1])), (int(landmarks[10][0]),int(landmarks[10][1])), (0, 255, 0), 2)  
  
    #main torse
    cv2.line(image, (int(landmarks[12][0]),int(landmarks[12][1])), (int(landmarks[11][0]),int(landmarks[11][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[12][0]),int(landmarks[12][1])), (int(landmarks[24][0]),int(landmarks[24][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[24][0]),int(landmarks[24][1])), (int(landmarks[23][0]),int(landmarks[23][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[11][0]),int(landmarks[11][1])), (int(landmarks[23][0]),int(landmarks[23][1])), (0, 255, 0), 2)  

    #legs
    cv2.line(image, (int(landmarks[24][0]),int(landmarks[24][1])), (int(landmarks[26][0]),int(landmarks[26][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[26][0]),int(landmarks[26][1])), (int(landmarks[28][0]),int(landmarks[28][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[23][0]),int(landmarks[23][1])), (int(landmarks[25][0]),int(landmarks[25][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[25][0]),int(landmarks[25][1])), (int(landmarks[27][0]),int(landmarks[27][1])), (0, 255, 0), 2)  
    
    #foot1
    cv2.line(image, (int(landmarks[28][0]),int(landmarks[28][1])), (int(landmarks[32][0]),int(landmarks[32][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[32][0]),int(landmarks[32][1])), (int(landmarks[30][0]),int(landmarks[30][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[28][0]),int(landmarks[28][1])), (int(landmarks[30][0]),int(landmarks[30][1])), (0, 255, 0), 2)  

    #foot2
    cv2.line(image, (int(landmarks[27][0]),int(landmarks[27][1])), (int(landmarks[29][0]),int(landmarks[29][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[29][0]),int(landmarks[29][1])), (int(landmarks[31][0]),int(landmarks[31][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[31][0]),int(landmarks[31][1])), (int(landmarks[27][0]),int(landmarks[27][1])), (0, 255, 0), 2)  

    #hand1
    cv2.line(image, (int(landmarks[12][0]),int(landmarks[12][1])), (int(landmarks[14][0]),int(landmarks[14][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[14][0]),int(landmarks[14][1])), (int(landmarks[16][0]),int(landmarks[16][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[16][0]),int(landmarks[16][1])), (int(landmarks[18][0]),int(landmarks[18][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[20][0]),int(landmarks[20][1])), (int(landmarks[18][0]),int(landmarks[18][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[20][0]),int(landmarks[20][1])), (int(landmarks[16][0]),int(landmarks[16][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[22][0]),int(landmarks[22][1])), (int(landmarks[16][0]),int(landmarks[16][1])), (0, 255, 0), 2)  

    #hand1
    cv2.line(image, (int(landmarks[11][0]),int(landmarks[11][1])), (int(landmarks[13][0]),int(landmarks[13][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[13][0]),int(landmarks[13][1])), (int(landmarks[15][0]),int(landmarks[15][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[15][0]),int(landmarks[15][1])), (int(landmarks[21][0]),int(landmarks[21][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[15][0]),int(landmarks[15][1])), (int(landmarks[19][0]),int(landmarks[19][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[19][0]),int(landmarks[19][1])), (int(landmarks[17][0]),int(landmarks[17][1])), (0, 255, 0), 2)  
    cv2.line(image, (int(landmarks[15][0]),int(landmarks[15][1])), (int(landmarks[17][0]),int(landmarks[17][1])), (0, 255, 0), 2)  


def pose_estimate(landmarks):
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

    angle = calculate_shoulder_angle(left_shoulder,right_shoulder)
    angle1 = calculate_angle(left_shoulder,right_shoulder,right_elbow)
    angle2 = calculate_angle(right_shoulder,left_shoulder,left_elbow)
            
    
            
    shoulder_posture = angle >= 0 and angle <= 10
    left_arm_posture = angle1 >= 90 and angle1 <= 124
    right_arm_posture = angle2 >= 90 and angle2 <= 124
    if shoulder_posture and left_arm_posture and right_arm_posture:
        stage = "Good"
        suggestion = None
    if shoulder_posture and not(left_arm_posture and right_arm_posture):
        stage="Bad"
        suggestion = "adjust your arm posture"
    if not(shoulder_posture) and (left_arm_posture and right_arm_posture):
        stage = "Bad"
        suggestion = "adjust your shoulder posture"
    if not(shoulder_posture) and not(left_arm_posture and right_arm_posture):
        stage = "Bad"
        suggestion = "adjust your posture"
        
    return [stage,angle,angle1,angle2]



if __name__ == "__main__":
    # Use OpenCV to capture video from the webcam
    model_path = "stress_detector.h5"
    model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})
    yolo_face = YOLO('face_detector.pt')
    person_model = YOLO(r"yolov8n.pt")
    frame = cv2.imread(r'test-images\test-image-1.jpg')
    cv2.resize(frame,(640,640),frame,cv2.INTER_LINEAR)
    with mp_pose.Pose(static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
        new_frame = frame
        face_results = yolo_face(frame)
        for r in face_results:
            boxes=r.boxes
            for box in boxes:
                x1,y1,x2,y2=box.xyxy[0]
                x1,y1,x2,y2=int(x1), int(y1), int(x2), int(y2)
                cv2.rectangle(frame,[x1,y1],[x2,y2],(255,0,0),2)
                cropped_frame = frame[y1:y2,x1:x2]
                processed_frame = preprocess_image(cropped_frame)
                prediction = model.predict(processed_frame)
                label = "not stressed" if prediction[0][0] > 0.5 else "stressed"
                cv2.putText(new_frame, label, (x1,y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        person_results = person_model(frame)
        for r in person_results:
            boxes = r.boxes
            for box in boxes:
                if box.cls[0]==0:
                    x,y,x1,y1=box.xyxy[0]
                    x,y,x1,y1=int(x), int(y), int(x1), int(y1)
                    cropped_image = frame[y:y1,x:x1]
                    cv2.rectangle(new_frame, (x,y), (x1,y1), (255,0,255),3)
                    cropped_image = np.ascontiguousarray(cropped_image)
                    cropped_image.flags.writeable = True
                    cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_RGB2BGR)
                    results = pose.process(cropped_image)
                    print(results)
                    try:
                        landmarks = results.pose_landmarks.landmark
                        adjusted_landmarks = [(lm.x * (x1-x) + x, lm.y * (y1-y) + y) for lm in results.pose_landmarks.landmark]
                        plot_pose(new_frame,adjusted_landmarks)
                        [stage,angle,angle1,angle2] = pose_estimate(landmarks)
                        color = (0,255,0)
                        if stage == "Bad":
                            color = (0,0,255)
                        cv2.putText(new_frame,str(round(angle1,3)),(int(adjusted_landmarks[12][0]),int(adjusted_landmarks[12][1])),cv2.FONT_HERSHEY_SIMPLEX, .5, color, 1, cv2.LINE_AA)
                        cv2.putText(new_frame,str(round(angle2,3)),(int(adjusted_landmarks[11][0]),int(adjusted_landmarks[11][1])),cv2.FONT_HERSHEY_SIMPLEX, .5, color, 1, cv2.LINE_AA)
                        cv2.putText(new_frame, stage, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
                    except Exception as e:
                        print(e)
                        pass

    cv2.imshow('frame', new_frame)
    cv2.waitKey(0)


0: 640x576 4 faces, 1266.1ms
Speed: 68.0ms preprocess, 1266.1ms inference, 241.3ms postprocess per image at shape (1, 3, 640, 576)


1/1 [==============================] - 0s 110ms/step



0: 640x576 2 persons, 1 chair, 329.6ms
Speed: 61.5ms preprocess, 329.6ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 576)


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>


<img src="results\combined_1.jpg" style="width:500px;height:500px">
<img src="results\combined_2.jpg" style="width:500px;height:500px">